This notebook is a step by step guide about how to train a deep neural network (DNN) in the DeepDeconv framework.

In [1]:
## Set up the sys.path in order to be able to import our modules
import os
import sys
module_path = os.path.abspath(os.path.join('../python'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('/data/alpha-transform'))
if module_path not in sys.path:
    sys.path.append(module_path)
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry

config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.75 

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Using TensorFlow backend.


We load the class of the network. This class must inherit from the DeepNet superclass. The method build_model has to be redefined in the child class with the wanted architecture. In our work, we use the network defined in deconvnNet. 

In [3]:
from DeepDeconv.deepnetFCS.DeconvNet_custom import DeconvNet

/home/cben-ali/Programs/deep-deconv/python/shape_constraint/cadmos_lib.py:149: ComplexWarning: Casting complex values to real discards the imaginary part
  coeff[s] = trafo.adjoint_transform(temp, do_norm=False)


# 1) Alexis net : gamma=0.01

In [7]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.01
shape_constraint=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_Target_Window_iter4'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)


Tensor("loss_1/conv2d_16_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_1:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_1:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_1/conv2d_16_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 96, 96, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 96, 96, 32)   128         conv2d_9[0][0]                   
_________________________________________

__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, 24, 24, 36)   144         concatenate_47[0][0]             
__________________________________________________________________________________________________
activation_56 (Activation)      (None, 24, 24, 36)   0           batch_normalization_56[0][0]     
__________________________________________________________________________________________________
separable_conv2d_50 (SeparableC (None, 24, 24, 12)   756         activation_56[0][0]              
__________________________________________________________________________________________________
concatenate_48 (Concatenate)    (None, 24, 24, 48)   0           concatenate_47[0][0]             
                                                                 separable_conv2d_50[0][0]        
__________________________________________________________________________________________________
batch_norm

In [6]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'new_targets/'+'image-*-multihdu_newtar.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(data_directory+'Target_Window/'+'Target*')
win_files.sort()
print(win_files)

SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-00-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-01-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-02-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-03-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-04-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-05-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-06-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-07-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-08-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/new_targets/image-shfl-09-multihdu_newtar.fits', '/data/DeepDeconv/data/vsc_eu

In [7]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet.hdf5",
             shearlet=True,gamma=1) 


Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma1_shearlet
Instructions for updating:
Colocations handled automatically by placer.
loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
Instructions for updating:
Use tf.cast instead.


In [6]:
dnn.shearlet_metric

<bound method DeepNet.shearlet_metric of <DeepDeconv.deepnetFCS.DeconvNet_custom.DeconvNet object at 0x7f6ef8a96f60>>

In [8]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_Window_iter4.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:399: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/20
5947/5947 [==============================] - 4263s 717ms/step - loss: 1.1116 - weighted_mean_squared_error: 0.6815 - weighted_shape_metric: 0.4301 - val_loss: 0.1755 - val_weighted_mean_squared_error: 0.0569 - val_weighted_shape_metric: 0.1186

Epoch 00001: val_loss improved from inf to 0.17551, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.01_Target_Window_iter4.hdf5
2019-08-29_10:12:51 - Epoch: 1, val_loss: 0.175511, val_weighted_mean_squared_error: 0.056892, val_weighted_shape_metric: 0.118619, loss: 1.112823, weighted_mean_squared_error: 0.682261, weighted_shape_metric: 0.430562
Epoch 2/20
5947/5947 [==============================] - 4206s 707ms/step - loss: 0.1387 - weighted_mean_squared_error: 0.0478 - weighted_shape_metric: 0.0909 - val_loss: 0.1102 - val_weighted_mean_squared_error: 0.0322 - val_weighted_shape_metric: 0.0780

Epoch 00002: val_loss improved from 0.17551 to 0.11020, saving model to ShapeNet2D_claire_sc4_layer4x5

5947/5947 [==============================] - 4158s 699ms/step - loss: 0.0375 - weighted_mean_squared_error: 0.0227 - weighted_shape_metric: 0.0148 - val_loss: 0.1135 - val_weighted_mean_squared_error: 0.0235 - val_weighted_shape_metric: 0.0900

Epoch 00016: val_loss did not improve from 0.06729
2019-08-30_03:50:58 - Epoch: 16, val_loss: 0.113479, val_weighted_mean_squared_error: 0.023499, val_weighted_shape_metric: 0.089980, loss: 0.037525, weighted_mean_squared_error: 0.022723, weighted_shape_metric: 0.014802
Epoch 17/20
5947/5947 [==============================] - 4191s 705ms/step - loss: 0.0370 - weighted_mean_squared_error: 0.0228 - weighted_shape_metric: 0.0142 - val_loss: 0.1161 - val_weighted_mean_squared_error: 0.0224 - val_weighted_shape_metric: 0.0936

Epoch 00017: val_loss did not improve from 0.06729
2019-08-30_05:00:49 - Epoch: 17, val_loss: 0.116058, val_weighted_mean_squared_error: 0.022410, val_weighted_shape_metric: 0.093648, loss: 0.037032, weighted_mean_squared_error

In [7]:
del dnn 

# 2) Alexis net : gamma=0.001

In [7]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.5
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)



loss (None, 96, 96)
temp (None, 1, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
loss (None, 96, 96)
temp (None, 1, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 96, 96, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_42 (BatchNo (None, 96, 96, 32)   128         conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_42 (Activation)      (None, 96, 96, 32)   0           batch_normalizatio

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.5_shearlet.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/20
5947/5947 [==============================] - 4034s 678ms/step - loss: 0.4784 - weighted_mean_squared_error: 0.4762 - weighted_shearlet_metric: 0.0022 - val_loss: 0.0348 - val_weighted_mean_squared_error: 0.0348 - val_weighted_shearlet_metric: 2.6633e-06

Epoch 00001: val_loss improved from inf to 0.03477, saving model to ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_with_shape_gamma0.5_shearlet.hdf5
2019-07-30_08:17:36 - Epoch: 1, val_loss: 0.034772, val_weighted_mean_squared_error: 0.034769, val_weighted_shearlet_metric: 0.000003, loss: 0.478969, weighted_mean_squared_error: 0.476775, weighted_shearlet_metric: 0.002193
Epoch 2/20
5947/5947 [==============================] - 3960s 666ms/step - loss: 0.0340 - 

 392/5947 [>.............................] - ETA: 1:48:20 - loss: 0.0279 - weighted_mean_squared_error: 0.0279 - weighted_shearlet_metric: 3.8946e-06

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma{4}_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

In [ ]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0.1
shearlet=True
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_with_shape_gamma1_shearlet'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shearlet=True)

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=20, batch_size=32,
                        model_file='', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode)

# Old target gamma=10^-4

In [12]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-0-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-1-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-10-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-11-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-12-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-13-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-14-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-15-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-16-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-17-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-18-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-19-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-2-mult

In [13]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0.0001) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs
Tensor("loss_3/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_3:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_3:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_3/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
[<tf.Tensor 'input_1_1:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'window_3:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'norm_3:0' shape=(?, 6, 1, 1) dtype=float32>]
['input_1', 'window', 'norm']


In [6]:
dnn2.gamma

0.0001

In [14]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.0001_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.0001_restart.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/10


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:363: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=10, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


5947/5947 [==============================] - 6783s 1s/step - loss: 0.1182 - weighted_mean_squared_error: 0.1179 - weighted_shape_metric: 2.3537e-04 - val_loss: 0.1199 - val_weighted_mean_squared_error: 0.1193 - val_weighted_shape_metric: 6.4551e-04

Epoch 00001: val_loss improved from inf to 0.11995, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.0001_restart.hdf5
2019-07-08_07:38:07 - Epoch: 1, val_loss: 0.119949, val_weighted_mean_squared_error: 0.119303, val_weighted_shape_metric: 0.000646, loss: 0.118161, weighted_mean_squared_error: 0.117925, weighted_shape_metric: 0.000235
Epoch 2/10
5947/5947 [==============================] - 5691s 957ms/step - loss: 0.1188 - weighted_mean_squared_error: 0.1186 - weighted_shape_metric: 1.8654e-04 - val_loss: 0.1178 - val_weighted_mean_squared_error: 0.1171 - val_weighted_shape_metric: 6.7807e-04

Epoch 00002: val_loss improved from 0.11995 to 0.11778, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_g

In [15]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar.hdf5",
             shape_constraint=True,gamma=0.1) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

Loading model...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar.hdf5
Renaming as...
ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs_newtar
Tensor("loss_4/conv2d_8_loss/strided_slice:0", shape=(96, 96), dtype=float32) Tensor("window_4:0", shape=(?, 96, 96, 1), dtype=float32) Tensor("norm_4:0", shape=(?, 6, 1, 1), dtype=float32) Tensor("loss_4/conv2d_8_loss/sub:0", shape=(?, 96, 96, ?), dtype=float32)
[<tf.Tensor 'input_1_3:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'window_4:0' shape=(?, 96, 96, 1) dtype=float32>, <tf.Tensor 'norm_4:0' shape=(?, 6, 1, 1) dtype=float32>]
['input_1', 'window', 'norm']


In [16]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart_newtar.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart_newtar.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Epoch 1/10
5947/5947 [==============================] - 5718s 961ms/step - loss: 1.1005 - weighted_mean_squared_error: 0.1737 - weighted_shape_metric: 0.9268 - val_loss: 1.6251 - val_weighted_mean_squared_error: 0.1509 - val_weighted_shape_metric: 1.4742

Epoch 00001: val_loss improved from inf to 1.62510, saving model to ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart_newtar.hdf5
2019-07-08_23:31:17 - Epoch: 1, val_loss: 1.625101, val_weighted_mean_squared_error: 0.150939, val_weighted_shape_metric: 1.474162, loss: 1.100902, weighted_mean_squared_error: 0.173756, weighted_shape_metric: 0.927145
Epoch 2/10
5947/5947 [==============================] - 5701s 959ms/step - loss: 0.4012 - weighted_mean_squa

# Appendix for GPU options

In [ ]:
## extra imports to set GPU options
#import tensorflow as tf
#from keras import backend as k

###################################
# TensorFlow wizardry
#config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
#config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5 

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Same with multiple GPUs, Tensorflow will pre-allocate the whole memory of all the GPUs.
Use the following to prevent it (only when your station has several GPUs like the SAPPCN63):

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd